In [0]:
pip install pyGPGO

In [0]:
pip install tsp_solver2

In [0]:
from pyGPGO.covfunc import squaredExponential
from pyGPGO.surrogates.GaussianProcess import GaussianProcess
from pyGPGO.surrogates.RandomForest import RandomForest
from pyGPGO.GPGO import GPGO
from pyGPGO.acquisition import Acquisition

import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
from tqdm import tqdm
from scipy.spatial import distance
import math
import operator
import time
import itertools
from datetime import datetime, timedelta
from tqdm import tqdm
from tsp_solver.greedy import solve_tsp
from scipy.spatial import distance_matrix
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import collections
from yellowbrick.cluster import KElbowVisualizer
import itertools
import requests

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
data = pd.read_csv("/content/drive/My Drive/Decision_Models/cities.csv")

## Funzione che individua le città Prime

In [0]:
def sieve_of_eratosthenes(n):
    primes = [True for i in range(n+1)] # Start assuming all numbers are primes
    primes[0] = False # 0 is not a prime
    primes[1] = False # 1 is not a prime
    for i in range(2,int(np.sqrt(n)) + 1):
        if primes[i]:
            k = 2
            while i*k <= n:
                primes[i*k] = False
                k += 1
    return(primes)
prime_cities = sieve_of_eratosthenes(max(data["CityId"]))

## Funzione Prime Swap

In [0]:
def prime_swap(sequenza):
  nnpath_with_primes = sequenza.copy()
  for index in range(20,len(nnpath_with_primes)-30):
      city = nnpath_with_primes[index]
      if (prime_cities[city] &  ((index+1) % 10 != 0)):        
          for i in range(-1,4):
              tmp_path = nnpath_with_primes.copy()
              swap_index = (int((index+1)/10) + i)*10 - 1
              tmp_path[swap_index],tmp_path[index] = tmp_path[index],tmp_path[swap_index]
              if total_distance_v2(tmp_path[min(swap_index,index) - 1 : max(swap_index,index) + 2]) < total_distance_v2(nnpath_with_primes[min(swap_index,index) - 1 : max(swap_index,index) + 2]):
                  nnpath_with_primes = tmp_path.copy() 
                  break
  return nnpath_with_primes

## Funzione che calcola la distanza totale

In [0]:
def total_distance_v2(path):
    prev_city = path[0]
    total_distance = 0
    step_num = 1
    for city_num in path[1:]:
        next_city = city_num
        total_distance = total_distance + \
            np.sqrt(pow((data["X"][city_num] - data["X"][prev_city]),2) + pow((data["Y"][city_num] - data["Y"][prev_city]),2)) * \
            (1+ 0.1*((step_num % 10 == 0)*int(not(prime_cities[prev_city]))))
        prev_city = next_city
        step_num = step_num + 1
    return total_distance

## Funzione che crea matrice di distanza utilizzando i dati di un solo cluster

In [0]:
def creaMatriceDistanza(datiCluster):
  c1 = []
  for i in datiCluster.index:
    c1.append(list(datiCluster[["X", "Y"]].loc[i]))
  dst = distance_matrix(c1,c1)
  return(dst)

In [0]:
def estraiPath(dati,path):
  lista=[]
  for i in range(len(path)):
    city = dati[dati['index']==path[i]]['CityId'].values
    lista.append(city[0])
  return(lista)

In [0]:
def ottienituttaSequenza(clusterCandidati, estremi_cluster):
  altri = estremi_cluster[estremi_cluster['Cluster'].isin(clusterCandidati)]
  path_cluster = []
  for i in range(len(altri)):
    path_cluster.append(altri.iloc[i]['sequenza'])
  path_cluster = list(itertools.chain.from_iterable(path_cluster))
  return(path_cluster)

In [0]:
def concatena(n,s,o,e):
 seq = np.append(n,s)
 seq2 = np.append(o,e)
 return(np.append(seq,seq2))

In [0]:
def ottieniPuntiEstremi(df):
  sud = df["Y"].sort_values().head(2).index.values
  nord = df["Y"].sort_values(ascending = False).head(2).index.values
  ovest = df["X"].sort_values(ascending=False).head(2).index.values
  est = df["X"].sort_values().head(2).index.values
  lista= [nord,sud,ovest,est]
  return(lista)

In [0]:
def mergeTour(ordineCluster, linkPunti):
  a=0
  poloNord = data[data['CityId']==0]
  tsp_cluster = pd.DataFrame(columns=['cluster','path_index','path_city','inizio','fine'])
  for i in tqdm(range(len(ordineCluster))):
    datiCluster = data[data['Cluster']==ordineCluster[i]]
    if (i==0):
      startIndex=0
      finishIndex = linkPunti[a][0]
    else:
      if (i==len(ordineCluster)-1):
        poloNord = data[data['CityId']==0]
        datiCluster = data[data['Cluster']==ordineCluster[-1]]
        datiCluster = pd.concat([poloNord, datiCluster],axis=0)
        startIndex=linkPunti[a][1]
        finishIndex=0
      else:
        startIndex = linkPunti[a][1]
        finishIndex = linkPunti[a+1][0]
        a=a+1
    datiCluster['index']=range(len(datiCluster))
    matrix = creaMatriceDistanza(datiCluster)
    index_start = datiCluster[datiCluster['CityId']==startIndex]['index'].values[0]
    index_end = datiCluster[datiCluster['CityId']==finishIndex]['index'].values[0]
    path = solve_tsp(matrix,endpoints=(index_start,index_end))
    path_city = estraiPath(datiCluster,path)
    tsp_cluster = tsp_cluster.append({'cluster': ordineCluster[i],
                      'path_index': path,
                      'path_city': path_city,
                      'inizio': path_city[0],
                      'fine': path_city[len(path_city)-1]},ignore_index=True)
  return(tsp_cluster)

## Funzione che crea matrice di distanza tra estremi cluster corrente e tutti gli altri estremi dei cluster candidati (non ancora visitati)

In [0]:
def ordineCluster_matriceDistanza(elementiCluster,elementiAltri):
  c1 = []
  c2 = []
  for i in elementiCluster.index:
    c1.append(list(elementiCluster[["X", "Y"]].loc[i]))
  for a in elementiAltri.index:
    c2.append(list(elementiAltri[["X", "Y"]].loc[a]))
  matrix_dst = distance_matrix(c1,c2)
  row = np.argwhere(matrix_dst==np.min(matrix_dst))[0][0]
  column = np.argwhere(matrix_dst==np.min(matrix_dst))[0][1]
  primo = elementiCluster.iloc[row]['CityId']
  secondo = elementiAltri.iloc[column]['CityId']
  clusterCandidato = int(elementiAltri.iloc[column]['Cluster'])
  coppia = [int(primo),int(secondo)]
  return(coppia, clusterCandidato)

## Funzione supporto
1.   ritorna ordine dei cluster e link per passare da un cluster ad un altro
2.   ritorna per ogni cluster il percorso ottimo TSP
3.   ritorna la distanza della sequenza totale ottenuta dopo unione dei percorsi di ogni cluster




In [0]:
def tourTsp(cluster_candidati, cluster_analisi, estremi_cluster):
  #Ordine di visita cluster e relativi collegamenti
  ordineCluster = []
  linkPunti = []
  utilizzati = []

  for i in range(len(cluster_analisi)-1):
    if i==0:
      cluster = data[data['CityId']==0]['Cluster'][0]
      ordineCluster.append(cluster)
    cluster_candidati = list(filter(lambda cluster_candidati: cluster_candidati!= cluster, cluster_candidati))
    seq_estremi_cluster = estremi_cluster[estremi_cluster['Cluster']==cluster]['sequenza'].values[0]
    elementi_cluster = data[data['CityId'].isin(seq_estremi_cluster)]
    if i > 0:
      elementi_cluster = elementi_cluster[elementi_cluster["CityId"]!=utilizzati[-1]]
    seq_estremi_cluster_altri = ottienituttaSequenza(cluster_candidati, estremi_cluster)
    elementi_cluster_altri = data[data['CityId'].isin(seq_estremi_cluster_altri)]
    punti, prossimo = ordineCluster_matriceDistanza(elementi_cluster,elementi_cluster_altri)
    linkPunti.append(punti)
    utilizzati.append(punti[0])
    utilizzati.append(punti[1])
    ordineCluster.append(prossimo)
    cluster = prossimo

  #Definisco cammino
  tsp_cluster = mergeTour(ordineCluster, linkPunti)

  #Ottenimento sequenza di visita dei punti
  path_cluster = []
  for i in range(len(tsp_cluster)):
    path_cluster.append(tsp_cluster.iloc[i]['path_city'])
  path_cluster = list(itertools.chain.from_iterable(path_cluster))
  path_cluster_ps = prime_swap(path_cluster)
  distance = 1/(total_distance_v2(path_cluster_ps))
  print("Distanza senza Prime Swap: "+str(total_distance_v2(path_cluster)))
  print("Distanza con Prime Swap: "+str(total_distance_v2(path_cluster_ps)))

  return(distance)

## Funzione principale che ritorna distanza del percorso ottenuto tramite clustering

In [0]:
def compute_distance(k):
  #Processo di clustering
  mclusterer = GaussianMixture(n_components=int(k), tol=0.01, random_state=66, verbose=1)
  cluster_label = mclusterer.fit_predict(data[['X','Y']])
  data["Cluster"] = cluster_label
  #Cluster polo Nord
  cluster = data['Cluster'].unique()
  clusterPoloNord = data[data['CityId']==0]['Cluster'][0]
  #Ottenimento punti estremi
  estremi_cluster = pd.DataFrame(columns=['Cluster','Nord','Sud','Ovest','Est'])
  for i in range(len(cluster)):
    dati_cluster = data[data['Cluster']==i]
    ris=ottieniPuntiEstremi(dati_cluster)
    estremi_cluster = estremi_cluster.append({'Cluster': i,
                                              'Nord': ris[0],
                                              'Sud': ris[1],
                                              'Ovest':ris[2],
                                              'Est': ris[3]},ignore_index=True)
  #Creazione della lista dei punti estremi
  estremi_cluster['sequenza'] = estremi_cluster.apply(lambda x: concatena(x['Nord'],x['Sud'],
                                                                        x['Ovest'],x['Est']),axis=1)
  #Ottenimento cluster candidati
  cluster_analisi = list(cluster)
  cluster_candidati = cluster_analisi

  distanza = tourTsp(cluster_candidati, cluster_analisi, estremi_cluster)
  return distanza

# AutoML

In [0]:
range_cluster = [5,100]
gp_modelsur = RandomForest()
fz_acquisition1=Acquisition(mode="ExpectedImprovement")

param ={'k': ('int',range_cluster)}

In [0]:
np.random.seed(2019)

SMBO_ei = GPGO(gp_modelsur,fz_acquisition1,compute_distance,param)
SMBO_ei.run(init_evals=5,max_iter=15)

In [0]:
def resultDataframe(k,dist):
  result = pd.DataFrame(k, columns=['k'])
  result['distance'] = 1/dist
  result['phase']="increment"
  result.loc[0:4,'phase']='initial'
  return result

In [0]:
risultati = resultDataframe(SMBO_ei.GP.X,SMBO_ei.GP.y)
risultati

# Metodo cluster con k specifico

In [0]:
compute_distance(39)